# An agentic implementation of multiple tools for function calling with Converse API using Mistral Large

In this Jupyter Notebook, we walkthrough an implementation of native function calling and agentic workflows with with the Converse API for Amazon Bedrock and Mistral Large. Function Calling is a powerful technique that allows large language models to connect to external tools, systems, or APIs to enable, which can be executed to perform actions based on user's input.

The Converse API is a unified structured text action for simplifying the invocations to Bedrock LLMs. It includes the possibility to define tools for implementing external functions that can be called or triggered from the LLMs.

---
## Mistral Model Selection

Today, Mistral Large supports native function calling with Converse API for Amazon Bedrock:

### 3. Mistral Large

- **Description:** A cutting-edge text generation model with top-tier reasoning capabilities. It can be used for complex multilingual reasoning tasks, including text understanding, transformation, and code generation.
- **Max Tokens:** 8,196
- **Context Window:** 32K
- **Languages:** English, French, German, Spanish, Italian
- **Supported Use Cases:** Synthetic Text Generation, Code Generation, RAG, or Agents

### Performance and Cost Trade-offs

The table below compares the model performance on the Massive Multitask Language Understanding (MMLU) benchmark and their on-demand pricing on Amazon Bedrock.

| Model           | MMLU Score | Price per 1,000 Input Tokens | Price per 1,000 Output Tokens |
|-----------------|------------|------------------------------|-------------------------------|
| Mistral Large | 81.2%      | \$0.008                   | \$0.024                     |

For more information, refer to the following links:

1. [Mistral Model Selection Guide](https://docs.mistral.ai/guides/model-selection/)
2. [Amazon Bedrock Pricing Page](https://aws.amazon.com/bedrock/pricing/)


---
## Supported parameters

The Mistral AI models have the following inference parameters.


```
{
    "prompt": string,
    "max_tokens" : int,
    "stop" : [string],    
    "temperature": float,
    "top_p": float,
    "top_k": int
}
```

The Mistral AI models have the following inference parameters:

- **Temperature** - Tunes the degree of randomness in generation. Lower temperatures mean less random generations.
- **Top P** - If set to float less than 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
- **Top K** - Can be used to reduce repetitiveness of generated tokens. The higher the value, the stronger a penalty is applied to previously present tokens, proportional to how many times they have already appeared in the prompt or prior generation.
- **Maximum Length** - Maximum number of tokens to generate. Responses are not guaranteed to fill up to the maximum desired length.
- **Stop sequences** - Up to four sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

---

### Local Setup (Optional)

For a local server, follow these steps to execute this jupyter notebook:

1. **Configure AWS CLI**: Configure [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) with your AWS credentials. Run `aws configure` and enter your AWS Access Key ID, AWS Secret Access Key, AWS Region, and default output format.

2. **Install required libraries**: Install the necessary Python libraries for working with SageMaker, such as [sagemaker](https://github.com/aws/sagemaker-python-sdk/), [boto3](https://github.com/boto/boto3), and others. You can use a Python environment manager like [conda](https://docs.conda.io/en/latest/) or [virtualenv](https://virtualenv.pypa.io/en/latest/) to manage your Python packages in your preferred IDE (e.g. [Visual Studio Code](https://code.visualstudio.com/)).

3. **Create an IAM role for SageMaker**: Create an AWS Identity and Access Management (IAM) role that grants your user [SageMaker permissions](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html). 

By following these steps, you can set up a local Jupyter Notebook environment capable of deploying machine learning models on Amazon SageMaker using the appropriate IAM role for granting the necessary permissions.

## Setup and Requirements

---
1. Create an Amazon SageMaker Notebook Instance - [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html)
    - For Notebook Instance type, choose ml.t3.medium.
2. For Select Kernel, choose [conda_python3](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-prepare.html).
3. Install the required packages.

<div class="alert alert-block alert-info"> 

<b>NOTE:

- </b> For <a href="https://aws.amazon.com/sagemaker/studio/" target="_blank">Amazon SageMaker Studio</a>, select Kernel "<span style="color:green;">Python 3 (ipykernel)</span>".

- For <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html" target="_blank">Amazon SageMaker Studio Classic</a>, select Image "<span style="color:green;">Base Python 3.0</span>" and Kernel "<span style="color:green;">Python 3</span>".

</div>

---

Before we start building the agentic workflow, we'll first install some libraries:

+ AWS Python SDKs [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to be able to submit API calls to [Amazon Bedrock](https://aws.amazon.com/bedrock/).
---

In [ ]:
%%writefile requirements.txt
boto3==1.34.125
pandas==2.2.2
 

Install required packages:

In [ ]:
!pip install -qU -r requirements.txt --quiet

In [ ]:
import boto3
from datetime import datetime
import json
import pandas as pd

Let's define a few variables and create a bedrock client.

In [ ]:
model_id = "mistral.mistral-large-2402-v1:0"
print(f"Using modelId: {model_id}")

region = "us-west-2"
print("Using region: ", region)

bedrock = boto3.client(service_name="bedrock-runtime", region_name=region)

## Load data

Let's say, we have a members dataset with health insurance member profiles, claims and plan information, . For more information about the sample dataset, please visit the documentation for [Mistral Function Calling](https://docs.mistral.ai/capabilities/function_calling/).

In [ ]:
def load_data() -> pd.DataFrame:
    """
    Load data from a JSON file into a Pandas DataFrame.

    Returns:
        pd.DataFrame: A Pandas DataFrame containing the data loaded from the JSON file.
        If an error occurs during file loading, an error message is returned instead.

    """
    local_path = "sample_data/members.json"

    try:
        df = pd.read_json(local_path)
        return df
    except (FileNotFoundError, ValueError) as e:
        return f"Error: {e}"

In [ ]:
df = load_data()
print(df)

---
## Function Calling

Function calling is the ability to reliably connect a large language model (LLM) to external tools and enable effective tool usage and interaction with external APIs. Mistral Large has the ability for building LLM powered chatbots or agents that need to retrieve context for the model or interact with external tools by converting natural language into API calls to retrieve specific domain knowledge. From conversational agents and math problem solving to API integration and information extraction, multiple use cases can benefit from this capability provided by Mistral Large.

---
## Tools

Function calling (Tool use) for Amazon Bedrock allow agents and large language models to interact with the world. In our example, we will define a tool list for a payment status (`get_payment_status`) and payment date (`get_payment_date`)  lookup tool. Note in our example we're just returning payment information from a sample dataset to illustrate the concept, but you could make it fully functional by connecting any database or service API.

In [ ]:
class ToolsList:
    # Define get member
    def get_member_profile(self, member_id) -> str:
        "Get Member profile by member Id"
        data = load_data()

        try:
            # Attempt to retrieve the profile for the member
            member_profile = data[member_id].profile
        except ValueError:
            # If the member ID is not found, return an error message
            return f"ERROR: Member ID {member_id} not found."

        # return the member Information
        result = f"Profile JSON for member  {member_id} is {member_profile}."
        print(f"Tool result: {result}")
        return result

    # Define get claims data tool function...
    def get_member_claims(self, member_id) -> str:
        "Get members claims by passing a member Id"
        data = load_data()

        try:
            # Attempt to retrieve the member claims
            claims = data[member_id].claims
        except ValueError:
            # If the member ID is not found, return an error message
            return f"ERROR: Member ID {member_id} not found."

        result = f"Claims for member in JSON format is  {member_id} is {claims}."
        print(f"Tool result: {result}")
        return result

---
### Tool configuration for Converse API

In this step, we structure our tools configuration for passing this information to our Converse API. We have to clearly define the schema that our tools are expecting in the corresponding functions.

**Note:** *With Converse API allows, we can define configurations to allow us either let the LLM choose automatically (auto) a tool, or overriding a fixed tool to be called always. To see more information, check the Bedrock Converse API documentation.*

In [ ]:
# Define the configuration for our tool...

toolConfig = {
    "tools": [],
    "toolChoice": {
        "auto": {},
        #'any': {},
        #'tool': {
        #    'name': 'get_member_profile'
        # }
    },
}

toolConfig["tools"].append(
    {
        "toolSpec": {
            "name": "get_member_profile",
            "description": "Get member profile for a given member id. use only when a member Id is known",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "member_id": {"type": "string", "description": "Member ID"}
                    },
                    "required": ["member_id"],
                }
            },
        }
    }
)

toolConfig["tools"].append(
    {
        "toolSpec": {
            "name": "get_member_claims",
            "description": "Get a given members claims information. Use only when a member Id is given and the query is about a specific member.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "member_id": {"type": "string", "description": "Member ID"}
                    },
                    "required": ["member_id"],
                }
            },
        }
    }
)

Let's define a handy function for calling Bedrock with the Converse API.

In [ ]:
# Function for caling the Bedrock Converse API...


def converse_with_tools(messages, system="", toolConfig=toolConfig):
    response = bedrock.converse(
        modelId=model_id, system=system, messages=messages, toolConfig=toolConfig
    )
    return response

We're now ready for setting up our orchestration flow. In this case, we'll make a first call to the LLM with the initial prompt from the user, and depending on the answer from the LLM we'll either call a tool (function calling) or end the interaction.

Note that in the case the LLM indicates that it wants to run a tool (function calling), it will give us the information required of tool name and arguments for us to run the relevant tool in our code; i.e. The LLMs cannot run the tools automatically.

In [ ]:
# Function for orchestrating the conversation flow...


def converse(prompt, system=""):
    # Add the initial prompt:
    messages = []
    messages.append({"role": "user", "content": [{"text": prompt}]})
    print(
        f"\n{datetime.now().strftime('%H:%M:%S')} - Initial prompt:\n{json.dumps(messages, indent=2)}"
    )

    # Invoke the model the first time:
    output = converse_with_tools(messages, system)
    print(
        f"\n{datetime.now().strftime('%H:%M:%S')} - Output so far:\n{json.dumps(output['output'], indent=2, ensure_ascii=False)}"
    )

    # Add the intermediate output to the prompt:
    messages.append(output["output"]["message"])

    function_calling = next(
        (
            c["toolUse"]
            for c in output["output"]["message"]["content"]
            if "toolUse" in c
        ),
        None,
    )

    # Check if function calling is triggered:
    if function_calling:
        # Get the tool name and arguments:
        tool_name = function_calling["name"]
        tool_args = function_calling["input"] or {}

        # Run the tool:
        print(
            f"\n{datetime.now().strftime('%H:%M:%S')} - Running ({tool_name}) tool..."
        )
        tool_response = getattr(ToolsList(), tool_name)(**tool_args) or ""
        if tool_response:
            tool_status = "success"
        else:
            tool_status = "error"

        # Add the tool result to the prompt:
        messages.append(
            {
                "role": "user",
                "content": [
                    {
                        "toolResult": {
                            "toolUseId": function_calling["toolUseId"],
                            "content": [{"text": tool_response}],
                            "status": tool_status,
                        }
                    }
                ],
            }
        )

        # Invoke the model one more time:
        output = converse_with_tools(messages, system)
        print(
            f"\n{datetime.now().strftime('%H:%M:%S')} - Final output:\n{json.dumps(output['output'], indent=2, ensure_ascii=False)}\n"
        )
    return

Now, we have everything setup and are ready for testing our function-calling bot.

Let's try with a few sample prompts, each one with different needs.

In [ ]:
prompts = [
    "What's the address of the member? Member Id is B4567891234",
    "What dates were the claims submitted,for member id B4567891234?",
    "How to submit an appeal for claim denial?",
]

for prompt in prompts:
    converse(
        system=[
            {
                "text": """You're provided with tools that can get themember profile and claims information for a given member ID; \
              use the tool only if required. Don't make reference to the tools in your final answer."""
            }
        ],
        prompt=prompt,
    )

As we can see, the LLM decides whether or not to call the tools depending on the question. The last question was answered without calling a tool.


## Distributors
- Amazon Web Services
- Mistral AI

---